# Explore U19 data Pipeline

Here we introduce some useful DataJoint tools to explore the U19 pipeline

First thing first, import datajoint

In [ ]:
import datajoint as dj

# 1. Browse schemas you have access to - `dj.list_schemas()`

In [ ]:
dj.list_schemas()


## Major schemas:   

Meta data: `u19_lab`, `u19_subject`, `u19_action`, `u19_task`  
Behavior: `u19_acquisition`, `u19_behavior` (also contains some imaging info)

These schemas have been generated by matlab or python, but as a user, you don't have to get the code to generate these tables. To work with these tables, the simplest thing you would do is to reconstruct a module that represents this schema with the method `create_virtual_module`.

In [ ]:
from u19_pipeline import lab

In [ ]:
lab.User()

In [ ]:
lab = dj.create_virtual_module('lab', 'u19_lab') # the first argument here is the __name__ of the virtual module
task = dj.create_virtual_module('task', 'u19_task') 
subject = dj.create_virtual_module('subject', 'u19_subject')
action = dj.create_virtual_module('action', 'u19_action')
acquisition = dj.create_virtual_module('acquisition', 'u19_acquisition')
behavior = dj.create_virtual_module('behavior', 'u19_behavior')

In [ ]:
task.Task()

In [ ]:
task.schema.save('task.py')

Now this is as if you have the code that generated the schema and tables, and have just imported them!

# 2. Browse tables in a schema - `dj.Diagram`

**Table tiers**:  
Manual table: green box  
Lookup table: gray box  
Imported table: blue oval  
Computed table: red circle  
Part table: plain text

**Dependencies**:  
One-to-one primary: thick solid line  
One-to-many primary: thin solid line  
Secondary foreign key reference: dashed line
renamed secondary foreign key references: orange dot

## Show tables in the whole schema

In [ ]:
dj.Diagram(subject)

## Show diagram of arbitruary parts of the database

A combination of arbitruary tables:

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(subject.Death) + dj.Diagram(subject.HealthStatus) + dj.Diagram(subject.Weaning)

In [ ]:
# same as above
dj.Diagram(subject.Subject) + subject.Death + subject.HealthStatus + subject.Weaning

The tables could be from different schemas:

In [ ]:
dj.Diagram(subject.Subject) + acquisition.Session

# 3. Getting the detailed definition of a table - `table.describe()`

In [ ]:
subject.Subject.describe();

In [ ]:
subject.Subject.heading

# 4. Browsing of data - queries

Query all subjects

In [ ]:
subject.Subject()

## Restriction  `&`: filtering data

### Restriction: Query one subject

In [ ]:
# restrict by string
subject.Subject & 'subject_nickname="B205"'

In [ ]:
# restrict by dictionary
subject.Subject & {'subject_nickname': 'B205'}

In [ ]:
subject.Subject & {'sex': 'Male'}

`OR` logic: a list of dictionaries

In [ ]:
subject.Subject & [{'user_id': 'hnieh'}, {'user_id': 'emanuele'}, 'line="DAT-IRES-CRE"']

In [ ]:
subject.Subject & "user_id in ('hnieh', 'emanuele')"

### Restriction: Query subjects born after a date

In [ ]:
subject.Subject & 'dob > "2021-01-01"'

### Restriction: subjects within a range of dates

In [ ]:
subject.Subject & 'dob between "2019-01-01" and "2019-04-01"'

### Restriction: Query subjects on multiple attributes

In [ ]:
subject.Subject & 'dob > "2019-01-01"' & 'sex="Male"'

### Restriction: Query subjects restricted by other tables

In [ ]:
# subjects that are dead
subject.Subject & subject.Death

In [ ]:
# subjects that are alive
subject.Subject - subject.Death

## Join `*`:  gather information from different tables

In [ ]:
action.WaterAdministration()

In [ ]:
subject.Subject * action.WaterAdministration

In [ ]:
action.WaterAdministration()

In [ ]:
lab.Lab()

In [ ]:
lab.Project()

Behavior of join:  
1. match the common field(s) of the primary keys in the two tables
2. do a combination of the non-matched part of the primary key
3. listing out the secondary attributes for each combination
4. if two tables have secondary attributes that share a same name, it will throw an error. To join, we need to rename that attribute for at least one of the tables.

For example,

In [ ]:
subject.Subject * acquisition.Session

## Projection `.proj()`: focus on attributes of interest

In [ ]:
(subject.Subject * acquisition.Session).proj()

In [ ]:
(subject.Subject * acquisition.Session).proj('dob', 'sex', 'location')

### rename attribute with ***proj()***

In [ ]:
subject.Subject.proj(gender='sex', birth_date='dob')  # 'sex->gender'

### perform simple computations with ***proj***

**Example 1: Get date of a weighing activity:**

In [ ]:
weighing_with_date = action.Weighing.proj(weighing_date='date(weighing_time)') # more options, check MySQL syntax

In [ ]:
weighing_with_date

**Example 2: Age of the animal when performing each session?**

In [ ]:
# First get the date of birth and the session date into the same query
q = subject.Subject * acquisition.Session
q = q.proj('dob')
q

In [ ]:
# Then compute the age
q_with_age = q.proj('dob', age='datediff(session_date, dob)') & 'dob is not NULL'
q_with_age

## Aggregation `.aggr()`: simple computation of one table against another table

Example: how many sessions does each subject do so far?

In [ ]:
subject.Subject.aggr(acquisition.Session, n='count(*)', lastest_session_date='max(session_date)')

# 5. Fetching data

## Fetch all fields: `fetch()`

In [ ]:
# fetch all data from a table
subjs = subject.Subject.fetch()
subjs

In [ ]:
subjs['subject_fullname']

In [ ]:
subjs['dob']

In [ ]:
# fetch as a list of dictionaries
subjs_dict = subject.Subject.fetch(as_dict=True)
subjs_dict

In [ ]:
# fetch as pandas dataframe
subjs_df = subject.Subject.fetch(format='frame').reset_index()
subjs_df

In [ ]:
# fetch the primary key
pk = subject.Subject.fetch('KEY')
pk

In [ ]:
# fetch specific attributes
dob, sex = subject.Subject.fetch('dob', 'sex')

In [ ]:
dob

In [ ]:
# fetch specific attributes as a list of dictionary
info = subject.Subject.fetch('dob', 'sex', as_dict=True)
info

## fetch data only from one entry: `fetch1`

In [ ]:
B205 = (subject.Subject & {'subject_nickname': 'B205'}).fetch1()  # "fetch1()" because we know there's only one

In [ ]:
B205

In [ ]:
B205_key = (subject.Subject & {'subject_nickname': 'B205'}).fetch1('KEY')

In [ ]:
B205_key

In [ ]:
B205_init_weight = (subject.Subject & {'subject_nickname': 'B205'}).fetch1('initial_weight')

In [ ]:
B205_init_weight

In [ ]:
(subject.Subject & {'subject_nickname': 'B205'}).fetch('initial_weight')